In [ ]:
# Install dependencies if not already installed (uncomment if necessary)
!pip install rdflib owlrl

In [ ]:
# You can paste below code in ChatGPT/Copilot and ask for a different example!

from rdflib import Graph, Namespace, RDF, RDFS, OWL
import owlrl

# Define namespaces
EX = Namespace("http://example.org/")
g = Graph()

g.bind("ex", EX)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("owl", OWL)

# Create classes
Person = EX.Person
Male = EX.Male
Female = EX.Female
Uncle = EX.Uncle

# Define properties
hasSibling = EX.hasSibling
hasChild = EX.hasChild

# Add classes to the graph
g.add((Person, RDF.type, OWL.Class))
g.add((Male, RDF.type, OWL.Class))
g.add((Female, RDF.type, OWL.Class))
g.add((Uncle, RDF.type, OWL.Class))

# Add class relationships (subClassOf)
g.add((Male, RDFS.subClassOf, Person))
g.add((Female, RDFS.subClassOf, Person))

# Define the Uncle class using OWL restrictions
# Uncle is defined as a Male who has a sibling who has a child
g.add((Uncle, RDFS.subClassOf, Male))  # Uncle is a Male
g.add((Uncle, OWL.equivalentClass, 
       EX.UncleRestriction))  # Equivalent to a restriction

# The restriction for Uncle: being a male whose sibling has a child
g.add((EX.UncleRestriction, RDF.type, OWL.Class))
g.add((EX.UncleRestriction, OWL.intersectionOf, EX.UncleConditionList))

# Condition 1: Must be Male
g.add((EX.UncleConditionList, RDF.first, Male))
g.add((EX.UncleConditionList, RDF.rest, EX.Condition2))

# Condition 2: Must have a sibling who has a child
g.add((EX.Condition2, RDF.first, EX.SiblingHasChildCondition))
g.add((EX.Condition2, RDF.rest, RDF.nil))

g.add((EX.SiblingHasChildCondition, RDF.type, OWL.Restriction))
g.add((EX.SiblingHasChildCondition, OWL.onProperty, hasSibling))
g.add((EX.SiblingHasChildCondition, OWL.someValuesFrom, 
       EX.HasChildRestriction))

g.add((EX.HasChildRestriction, RDF.type, OWL.Restriction))
g.add((EX.HasChildRestriction, OWL.onProperty, hasChild))
g.add((EX.HasChildRestriction, OWL.someValuesFrom, Person))

# Create individuals
john = EX.John  # John is Male
mary = EX.Mary  # Mary is Female
jane = EX.Jane  # Jane is Female
bob = EX.Bob    # Bob is Male (child of Mary)

# Assign types
g.add((john, RDF.type, Male))
g.add((mary, RDF.type, Female))
g.add((jane, RDF.type, Female))
g.add((bob, RDF.type, Male))

# Define sibling relationships
g.add((john, hasSibling, mary))  # John and Mary are siblings
g.add((mary, hasChild, bob))     # Mary has a child, Bob

# Perform reasoning with OWL RL
owlrl.DeductiveClosure(owlrl.OWLRL_Semantics).expand(g)

# Show all triples after reasoning, filtered for Uncle class inferences
for s, p, o in g:
    if o == Uncle:
        print(f"Inferred: {s} is an {o}")
